In [102]:
import pandas as pd
import numpy as np
import os 
import re
import ast
pd.options.display.max_columns = None

Cargaremos los dos ficheros de datos

In [103]:
df_limp = pd.read_csv("attacks_limpieza_5.csv", index_col=0)
df_limp.head(2)

,case_number,type,country,area,location,activity,name,sex_,age,time,species_,href,fatal_(y/n),injury,date,sex_mode,type_mode,year,age_new
0,1800.00.00,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, all onboard were killed by sharks",1800,F,Unprovoked,1927.133059,27.267683
1,1797.05.28.R,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,FATAL,Reported May-28-1797,M,Unprovoked,1927.133059,27.267683


In [104]:
clima = pd.read_csv("etl_1.csv", index_col = 0)
clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA


In [105]:
clima.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            54, 55, 56, 57, 58, 59, 60, 61, 62, 63],
           dtype='int64', length=320)

 - Reseteamos el index para que no se repitan valores y así podamos hacer el siguiente paso sin errores.

In [106]:
clima.reset_index(inplace=True)

 - Eliminamos la antigua columna 'index'.

In [107]:
clima.drop('index', inplace=True, axis=1)

In [108]:
clima.shape

(320, 17)

Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer:
 - USA
 - Australia
 - New Zealand
 - South Africa
 - Papua New Guinea

In [109]:
filtrado_paises = df_limp[(df_limp['country'] == 'australia') 
                  |(df_limp['country'] == 'usa')|(df_limp['country'] == 'south africa') | (df_limp['country'] == 'papua new guinea')
                  | (df_limp['country'] == 'new zealand')]

 - Groupby de rh_profile

In [111]:
clima['rh_profile'] = clima['rh_profile'].apply(ast.literal_eval)

In [112]:
len(clima['rh_profile'][0])

16

In [113]:
rh = clima['rh_profile'].apply(pd.Series)

In [114]:
rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 13}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -2}"
1,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -1}"


In [115]:
for i in range(len(rh.columns)): 

    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    valores = list(rh[i].apply(pd.Series)["rh"] )
    clima.insert(i, nombre, valores)

In [116]:
clima.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,13,13,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-2,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,14,12,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA


 - Groupby de wind_profile

In [117]:
clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

In [118]:
x = clima['wind_profile'].apply(pd.Series)

In [119]:
x.sample()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
58,"{'layer': '950mb', 'direction': 90, 'speed': 4}","{'layer': '900mb', 'direction': 100, 'speed': 5}","{'layer': '850mb', 'direction': 100, 'speed': 5}","{'layer': '800mb', 'direction': 100, 'speed': 4}","{'layer': '750mb', 'direction': 105, 'speed': 3}","{'layer': '700mb', 'direction': 110, 'speed': 3}","{'layer': '650mb', 'direction': 80, 'speed': 3}","{'layer': '600mb', 'direction': 60, 'speed': 3}","{'layer': '550mb', 'direction': 20, 'speed': 3}","{'layer': '500mb', 'direction': 355, 'speed': 5}","{'layer': '450mb', 'direction': 345, 'speed': 5}","{'layer': '400mb', 'direction': 340, 'speed': 5}","{'layer': '350mb', 'direction': 200, 'speed': 3}","{'layer': '300mb', 'direction': 245, 'speed': 5}","{'layer': '250mb', 'direction': 255, 'speed': 5}","{'layer': '200mb', 'direction': 260, 'speed': 7}"


In [120]:
for i in range(len(x.columns)): 

    nombre = "layer" + str(x[i].apply(pd.Series)["layer"][0]) 
    valores = list(x[i].apply(pd.Series)["speed"] )
    clima.insert(i, nombre, valores)

In [121]:
clima.head(2)

,layer950mb,layer900mb,layer850mb,layer800mb,layer750mb,layer700mb,layer650mb,layer600mb,layer550mb,layer500mb,layer450mb,layer400mb,layer350mb,layer300mb,layer250mb,layer200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,4,3,3,3,2,2,2,3,3,3,3,4,3,3,3,4,13,13,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-2,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,4,4,3,3,2,2,2,2,3,3,3,4,3,3,3,3,14,12,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA


 - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por pais.

 - Guardar los resultados obtenidos en un csv.
